In [39]:
import numpy as np 
import pandas as pd 
import gmaps 
from vincenty import vincenty as vc
from tqdm.notebook import tqdm
from query_heatmap import QueryHeatMap

In [40]:
# Params
number_of_cells = 100
rows=1000000
dist_cells = 0.1

In [41]:
#Lyon Map Coordinates

# MAX Point
meximiax_lat = 45.904791
meximiax_long = 5.186973

# Min Point
granger_lat = 45.524679
granger_long = 4.642500


dist_lat = vc((meximiax_lat,meximiax_long),(granger_lat,meximiax_long))
dist_long = vc((meximiax_lat,meximiax_long),(meximiax_lat,granger_long))
"""
+-----------+
|           |
|           |
|           |
|           |
+-----------+
"""
number_of_long_cells = round(dist_long / dist_cells)
number_of_lat_cells  = round(dist_lat / dist_cells)


long_cell_size = (meximiax_long - granger_long)/number_of_long_cells
lat_cell_size = (meximiax_lat - granger_lat)/number_of_lat_cells

LONG_CELLS = np.arange(granger_long, meximiax_long, long_cell_size)
LAT_CELLS = np.arange(granger_lat, meximiax_lat, lat_cell_size) 
long_cell_size, lat_cell_size




(0.0012902203791469195, 0.0009007393364929005)

In [42]:
def filter_rows(row, minLat, minLong, maxLat, maxLong) :
    return ( (minLat<row['lat']) & (maxLat>row['lat']) & (minLong<row['long']) & (maxLong>row['long']))    

In [43]:
# We cut the csv in small chunks
iter_csv = pd.read_csv("./privamov/privamov-gps",
                   sep='\t',
                   names=['id','timestamp','long','lat'],
                   header = None,
                   infer_datetime_format= True,
                   iterator=True,
                   chunksize=1000)

In [ ]:
# Here we load the chunks and filter the dataset before loading in pandas
minLat = min(meximiax_lat, granger_lat)
minLong = min(meximiax_long,granger_long)
maxLat = max(meximiax_lat,granger_lat)
maxLong  = max(meximiax_long, granger_long)
data = pd.concat([chunk[filter_rows(chunk, minLat, minLong, maxLat, maxLong)] for chunk in  tqdm(iter_csv)])

In [33]:
from typing import List
printemps_week_day = []
printemps_week_night = [] 
printemps_wend_night = []
printemps_wend_day = []
ete_week_day = []
ete_week_night = []
ete_wend_day = []
ete_wend_night = []
automne_week_day = []
automne_week_night = []
automne_wend_day = []
automne_wend_night = []
hiver_week_day = []
hiver_week_night = []
hiver_wend_day = []
hiver_wend_night = []

def add_to_correct_list(index,night: bool,week: bool,season: int):
    if season == 1: 
        if week: 
            if night:
                printemps_week_day.append(index)
            else:
                printemps_week_night.append(index)
        else: 
            if night:
                printemps_wend_day.append(index)
            else:
                printemps_wend_night.append(index)
    elif season == 2: 
        if week: 
            if night:
                ete_week_day.append(index)
            else:
                ete_week_night.append(index)
        else: 
            if night:
                ete_wend_day.append(index)
            else:
                ete_wend_night.append(index)
    elif season == 3:
        if week: 
            if night:
                automne_week_day.append(index)
            else:
                automne_week_night.append(index)
        else: 
            if night:
                automne_wend_day.append(index)
            else:
                automne_wend_night.append(index)
    elif season == 4: 
        if week: 
            if night:
                hiver_week_day.append(index)
            else:
                hiver_week_night.append(index)
        else: 
            if night:
                hiver_wend_day.append(index)
            else:
                hiver_wend_night.append(index)

In [34]:
data

,id,timestamp,long,lat
160803,1,2014-10-07 12:30:19.611,4.869205,45.770287
160804,1,2014-10-08 07:14:04.115,4.869438,45.772273
160805,1,2014-10-08 07:14:04.135,4.869438,45.772273
160806,1,2014-10-08 07:14:05.105,4.869597,45.772993
160807,1,2014-10-08 07:14:05.124,4.869597,45.772993
...,...,...,...,...
996980,8,2014-11-28 04:55:56.683,5.079448,45.716740
996983,8,2014-11-28 04:55:57.651,5.079447,45.716748
996984,8,2014-11-28 04:55:57.664,5.079447,45.716748
996987,8,2014-11-28 04:55:58.654,5.079447,45.716758


In [35]:
i = 0
for index, row in  tqdm(data.iterrows()): 
    date = pd.to_datetime(row["timestamp"], yearfirst=True).to_pydatetime()
    if (False): 
        pass 
    else: 
        night = False
        week = False
        season = 1
        if (date.hour) > 18: 
            night = True
        # week-end
        if (date.day > 5):
            week = True
        # Season    
        month = date.month
        if (month < 3): 
            season = 1
        elif (month < 6): 
            season = 2
        elif (month < 9):
            season =3 
        else: 
            season = 4 
            
    add_to_correct_list(i,night,week,season)
    i += 1

In [36]:
to_file = { 
"printemps/week/day/" : printemps_week_day ,
"printemps/week/night/" : printemps_week_night ,
"printemps/wend/night/" : printemps_wend_night ,
"printemps/wend/day/" : printemps_wend_day ,
"ete/week/day/" : ete_week_day ,
"ete/week/night/" : ete_week_night ,
"ete/wend/day/" : ete_wend_day ,
"ete/wend/night/" : ete_wend_night ,
"automne/week/day/" : automne_week_day ,
"automne/week/night/" : automne_week_night ,
"automne/wend/day/" : automne_wend_day ,
"automne/wend/night/" : automne_wend_night ,
"hiver/week/day/" : hiver_week_day ,
"hiver/week/night/" : hiver_week_night ,
"hiver/wend/day/" : hiver_wend_day ,
"hiver/wend/night/" : hiver_wend_night ,
}

In [37]:
def generate_heat_map(indices: List, data):
    """Generates a heat_map for a given list of indices of data.

    ----------
    indices : List
        indices of df to add to heatmap 
    data: pd.Dataframe 
        dataframe storing gps values
    """
    # We create an array of 0s of number_of_cells x number_of_cells to store the ppl mouves.
    ppl_counts = np.zeros((number_of_long_cells,number_of_lat_cells)) 
    for index in tqdm(indices):
        lat = data['lat'].iloc[index]
        lng =  data['long'].iloc[index] 
        for lat_index,lat_cell in enumerate(LAT_CELLS):
            if (lat_cell- lat_cell_size <= lat and lat < lat_cell+ lat_cell_size):
                for long_index,long_cell in enumerate(LONG_CELLS):
                    if (long_cell - long_cell_size <= lng and lng < long_cell + long_cell_size):
                        ppl_counts[lat_index,long_index] += 1
    return ppl_counts

In [38]:
import os
for filename, indices in to_file.items(): 
    os.makedirs(filename, exist_ok = True)
    if len(indices):
        print(f"Processing: {filename}")
        ppl_counts = generate_heat_map(indices,data)
        np.save(f"{filename}data.npy", ppl_counts)

KeyboardInterrupt: 